In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize, sent_tokenize


let's start with loading the dataset and storing them as dataframes.

In [2]:
gpt3 = pd.read_csv("data\\gpt3.csv")
instruct = pd.read_csv("data\\instruct.csv")
new_ctrl = pd.read_csv("data\\new_ctrl.csv")
new_fair = pd.read_csv("data\\new_fair.csv")
new_gpt = pd.read_csv("data\\new_gpt.csv")
new_gpt2 = pd.read_csv("data\\new_gpt2.csv")
new_grover = pd.read_csv("data\\new_grover.csv")
new_human = pd.read_csv("data\\new_human.csv")
new_pplm = pd.read_csv("data\\new_pplm.csv")
new_xlm = pd.read_csv("data\\new_xlm.csv")
new_xlnet = pd.read_csv("data\\new_xlnet.csv")

In [3]:
new_ctrl.head()

,Unnamed: 0,Generation,label
0,0,latest headlines on cnn business tl;dr the u.s...,ctrl
1,1,china wants to take a victory lap over its han...,ctrl
2,2,coronavirus disinformation creates challenges ...,ctrl
3,3,china coronavirus: eating wild animals made il...,ctrl
4,4,china's economy could shrink for the first tim...,ctrl


In [10]:
new_ctrl.iloc[ : , 1:]

,Generation,label
0,latest headlines on cnn business tl;dr the u.s...,ctrl
1,china wants to take a victory lap over its han...,ctrl
2,coronavirus disinformation creates challenges ...,ctrl
3,china coronavirus: eating wild animals made il...,ctrl
4,china's economy could shrink for the first tim...,ctrl
...,...,...
1061,how much of your body is your own? a. i think ...,ctrl
1062,how do you keep a space station clean? q. what...,ctrl
1063,the city where you pay a year's rent up front ...,ctrl
1064,the bbc app gives you the best of bbc wherever...,ctrl


In [11]:
gpt3.head()

,Unnamed: 0,prompt,Generation,label
0,0,Latest Headlines on CNN Business,"1. Ford recalling 373,000 cars in the US over ...",GPT3
1,1,China wants to take a victory lap over its han...,"October 27, 2016, 10:06 p.m.China has issued a...",GPT3
2,2,Coronavirus disinformation creates challenges ...,The Chinese government has been vocal about it...,GPT3
3,3,China coronavirus: Eating wild animals made il...,The World Health Organization (WHO) says that ...,GPT3
4,4,China's economy could shrink for the first tim...,A new study by the Oxford Economics research f...,GPT3


In [4]:
gpt3 = gpt3.iloc[ : , 2:]
instruct = instruct.iloc[ : , 2:]
new_ctrl = new_ctrl.iloc[ : , 1:]
new_fair = new_fair.iloc[ : , 1:]
new_gpt = new_gpt.iloc[ : , 1:]
new_gpt2 = new_gpt2.iloc[ : , 1:]
new_grover = new_grover.iloc[ : , 1:]
new_human = new_human.iloc[ : , 1:]
new_pplm = new_pplm.iloc[ : , 1:]
new_xlm = new_xlm.iloc[ : , 1:]
new_xlnet = new_xlnet.iloc[ : , 1:]

In [5]:
gpt3['Generation'][20]

"Mount Everest, the world's highest mountain, was closed to climbers Thursday due to fears of a coronavirus pandemic."

In [6]:
gpt3.head()

,Generation,label
0,"1. Ford recalling 373,000 cars in the US over ...",GPT3
1,"October 27, 2016, 10:06 p.m.China has issued a...",GPT3
2,The Chinese government has been vocal about it...,GPT3
3,The World Health Organization (WHO) says that ...,GPT3
4,A new study by the Oxford Economics research f...,GPT3


In [15]:
def clean_the_text(data):
    clean_data = []
    for i in list(data['Generation']):
        # Replace all the specified patterns with spaces
        text = re.sub('[^A-Za-z0-9]+|<unk>|\n| \ |<UNK>|News|<eos>|<eod>|<eop>', ' ', str(i))
        # Replace consecutive whitespaces with a single space
        text = re.sub('\s+', ' ', text)
        # Remove leading/trailing whitespaces
        text = text.strip()
        clean_data.append(text)
    
    # Create a new DataFrame with cleaned texts and original labels
    return clean_data

In [16]:
gpt3['Generation'] = clean_the_text(gpt3)
instruct['Generation'] = clean_the_text(instruct)
new_ctrl['Generation'] = clean_the_text(new_ctrl)
new_fair['Generation'] = clean_the_text(new_fair)
new_gpt['Generation'] = clean_the_text(new_gpt)
new_gpt2['Generation'] = clean_the_text(new_gpt2)
new_grover['Generation'] = clean_the_text(new_grover)
new_pplm['Generation'] = clean_the_text(new_pplm)
new_xlm['Generation'] = clean_the_text(new_xlm)
new_xlnet['Generation'] = clean_the_text(new_xlnet)

Statistics

In [46]:
def statistics(data):
    word_lengths = []
    sentence_lengths = []
#     data = data['Generation']
    # Tokenize each text in the data
    for text in data:
#         print(text)
        
        
        words = word_tokenize(text)
        if len(words) == 0:
            continue
        word_lengths.append(len(words))
        sentences = sent_tokenize(text)
        if len(sentences) == 0:
            continue
        sentence_lengths.append(len(sentences))
#     print(word_lengths)
#     print(sentence_lengths)
    
    # Filter out any lengths that are more than 2 standard deviations away from the mean
    mean_word_length = np.mean(word_lengths)
    std_word_length = np.std(word_lengths)
    new_word_lengths = [x for x in word_lengths if (x > mean_word_length - 2 * std_word_length)]
    new_word_lengths = [x for x in new_word_lengths if (x < mean_word_length + 2 * std_word_length)]
    
    mean_sentence_length = np.mean(sentence_lengths)
    std_sentence_length = np.std(sentence_lengths)
    new_sentence_lengths = [x for x in sentence_lengths if (x > mean_sentence_length - 2 * std_sentence_length)]
    new_sentence_lengths = [x for x in new_sentence_lengths if (x < mean_sentence_length + 2 * std_sentence_length)]
    
    # Return a DataFrame with the statistics
    return pd.DataFrame({'Avg_word': np.mean(new_word_lengths), 
                         'SD_word': np.std(new_word_lengths),
                         'Avg_sent': np.mean(new_sentence_lengths),
                         'SD_sent': np.std(new_sentence_lengths)}, index=[0])

In [47]:
import numpy as np
gpt3 = gpt3.replace(np.nan,' ',regex=True)

statistics(gpt3['Generation'])

,Avg_word,SD_word,Avg_sent,SD_sent
0,97.340932,72.792486,3.385602,2.590803


In [48]:
import numpy as np
instruct = instruct.replace(np.nan,' ',regex=True)

statistics(instruct['Generation'])

,Avg_word,SD_word,Avg_sent,SD_sent
0,133.629191,69.392365,5.165029,2.696187


In [49]:
import numpy as np
# instruct = instruct.replace(np.nan,' ',regex=True)

statistics(new_ctrl['Generation'])

,Avg_word,SD_word,Avg_sent,SD_sent
0,528.272115,69.105285,30.466143,16.258389


In [52]:
new_fair_stats = statistics(new_fair['Generation'])
new_gpt_stats = statistics(new_gpt['Generation'])
new_gpt2_stats = statistics(new_gpt2['Generation'])
new_grover_stats = statistics(new_grover['Generation'])
new_human_stats = statistics(new_human['Generation'])
new_pplm_stats = statistics(new_pplm['Generation'])
new_xlm_stats = statistics(new_xlm['Generation'])
new_xlnet_stats = statistics(new_xlnet['Generation'])

In [53]:
new_fair_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,450.260954,24.602548,19.835481,4.641423


In [54]:
new_gpt_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,431.950147,8.786401,33.581188,4.88986


In [55]:
new_gpt2_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,452.716814,28.46783,20.632278,6.944668


In [56]:
new_grover_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,552.016053,196.044181,21.407336,8.911193


In [57]:
new_human_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,830.470302,789.914863,33.546251,32.815327


In [58]:
new_pplm_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,471.462572,19.366528,15.320717,4.387224


In [59]:
new_xlm_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,445.492567,20.003473,3.815739,1.256317


In [60]:
new_xlnet_stats

,Avg_word,SD_word,Avg_sent,SD_sent
0,491.721068,13.047556,4.707956,1.562819


In [39]:
gpt3.head()

,Generation,label
0,"1. Ford recalling 373,000 cars in the US over ...",GPT3
1,"October 27, 2016, 10:06 p.m.China has issued a...",GPT3
2,The Chinese government has been vocal about it...,GPT3
3,The World Health Organization (WHO) says that ...,GPT3
4,A new study by the Oxford Economics research f...,GPT3


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\girid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Combined Dataset for Task3

In [62]:
df = pd.concat([new_ctrl,new_fair, new_gpt, new_gpt2, new_grover, new_xlm, new_xlnet, new_pplm, new_human, gpt3, instruct])


In [63]:
len(df)

11726

In [64]:
df = df.sample(len(df))

In [65]:
len(df)

11726

In [90]:
df['label'] = df['label'].apply(str.lower)

In [93]:
df['label_int'] = pd.Categorical(df['label']).codes

In [92]:
l

array(['human', 'instructgpt', 'pplm', 'gpt', 'xlm', 'xlnet', 'gpt2',
       'fair', 'ctrl', 'gpt3', 'grover'], dtype=object)

In [94]:
df.head(30)

,Generation,label,class_categorical,label_int
0,Local News get the news that's local to you,human,7,6
1,:The number of confirmed cases of coronavirus ...,instructgpt,8,7
2,| | how ancient egyptian cosmetics influenced ...,pplm,9,8
3,"is it allergies, the flu or the coronavirus? h...",gpt,4,2
4,don't toss this food scrap (it could be vastly...,xlm,10,9
5,a coronavirus outbreak could be devastating fo...,xlnet,11,10
6,i forgave my father for walking out on me. i h...,xlm,10,9
7,things to do indoors with your children the fo...,gpt2,5,3
8,coronavirus: english local elections postponed...,fair,3,1
9,"split by trump's travel ban, a family races to...",ctrl,2,0


In [95]:
df = df.reset_index()
df = df[['Generation','label','label_int']]


In [96]:
df.to_csv('Task3.csv', index = False, header = True)

In [100]:
df.head(30)

,Generation,label,label_int
0,Local News get the news that's local to you,human,6
1,:The number of confirmed cases of coronavirus ...,instructgpt,7
2,| | how ancient egyptian cosmetics influenced ...,pplm,8
3,"is it allergies, the flu or the coronavirus? h...",gpt,2
4,don't toss this food scrap (it could be vastly...,xlm,9
5,a coronavirus outbreak could be devastating fo...,xlnet,10
6,i forgave my father for walking out on me. i h...,xlm,9
7,things to do indoors with your children the fo...,gpt2,3
8,coronavirus: english local elections postponed...,fair,1
9,"split by trump's travel ban, a family races to...",ctrl,0


## Random Forest with TF-IDF vectors

In [101]:
import numpy as np
import pandas as pd 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics


In [102]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [103]:
from yellowbrick.classifier import ClassPredictionError
from sklearn.metrics import classification_report

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [104]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter, FixedLocator
import matplotlib


In [108]:
def classify(data, label):
            
    
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
#     clf = LogisticRegression(solver = 'lbfgs',multi_class='auto')
    
    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]

    test_corpus = X_test
#     test_corpus = [" ".join(x) for x in test_corpus]

#     ax = axes.set_ylim([0,300])
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
#     # Instantiate the classification model and visualizer
#     visualizer = ClassPredictionError(clf, classes= ['human', 'ctrl', 'gpt', 'gpt2', 'grover', 'xlm', 'xlnet', 'pplm', 'fair'])

#     # Fit the training data to the visualizer
#     visualizer.fit(train_vector, y_train)

#     # Evaluate the model on the test data
#     visualizer.score(test_vector, y_test)

#     # Draw visualization
#     visualizer.show()
    
    l=['ctrl','fair' ,'gpt', 'gpt2', 'gpt3', 'grover', 'human', 'instructgpt', 'pplm', 'xlm', 'xlnet']
    matrix = confusion_matrix(y_test, pred, labels = l)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = l,
                                digits=4))
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

In [110]:
rd, v = classify(df['Generation'], df['label'])


              precision    recall  f1-score   support

        ctrl     0.9906    0.9906    0.9906       213
        fair     0.4640    0.5446    0.5011       213
         gpt     0.9860    0.9953    0.9907       213
        gpt2     0.4906    0.4860    0.4883       214
        gpt3     0.5015    0.7840    0.6117       213
      grover     0.6439    0.6168    0.6301       214
       human     0.9429    0.6168    0.7458       214
 instructgpt     0.7081    0.6150    0.6583       213
        pplm     0.8036    0.6338    0.7087       213
         xlm     1.0000    1.0000    1.0000       213
       xlnet     0.9906    0.9859    0.9882       213

    accuracy                         0.7515      2346
   macro avg     0.7747    0.7517    0.7558      2346
weighted avg     0.7746    0.7515    0.7556      2346

confusion matrix:  [0.99061033 0.54460094 0.99530516 0.48598131 0.78403756 0.61682243
 0.61682243 0.61502347 0.63380282 1.         0.98591549]
Accuracy: 0.7514919011082694
Recall:  0.7517

## Logistic Regression multi class with TF-IDF vectors

In [111]:
def classify(data, label):
            
    
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
#     clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

#     clf = LogisticRegression(solver = 'lbfgs',multi_class='auto')
    
    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]

    test_corpus = X_test
#     test_corpus = [" ".join(x) for x in test_corpus]

#     ax = axes.set_ylim([0,300])
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
#     # Instantiate the classification model and visualizer
#     visualizer = ClassPredictionError(clf, classes= ['human', 'ctrl', 'gpt', 'gpt2', 'grover', 'xlm', 'xlnet', 'pplm', 'fair'])

#     # Fit the training data to the visualizer
#     visualizer.fit(train_vector, y_train)

#     # Evaluate the model on the test data
#     visualizer.score(test_vector, y_test)

#     # Draw visualization
#     visualizer.show()
    
    l=['ctrl','fair' ,'gpt', 'gpt2', 'gpt3', 'grover', 'human', 'instructgpt', 'pplm', 'xlm', 'xlnet']
    matrix = confusion_matrix(y_test, pred, labels = l)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = l,
                                digits=4))
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

In [112]:
lr, v = classify(df['Generation'], df['label'])

              precision    recall  f1-score   support

        ctrl     1.0000    0.9953    0.9976       213
        fair     0.5660    0.5634    0.5647       213
         gpt     0.9726    1.0000    0.9861       213
        gpt2     0.5194    0.5000    0.5095       214
        gpt3     0.4783    0.5681    0.5193       213
      grover     0.5673    0.6495    0.6057       214
       human     0.6905    0.5421    0.6073       214
 instructgpt     0.6367    0.7324    0.6812       213
        pplm     0.8773    0.6714    0.7606       213
         xlm     0.9861    1.0000    0.9930       213
       xlnet     1.0000    0.9718    0.9857       213

    accuracy                         0.7447      2346
   macro avg     0.7540    0.7449    0.7464      2346
weighted avg     0.7538    0.7447    0.7462      2346

confusion matrix:  [0.99530516 0.56338028 1.         0.5        0.56807512 0.64953271
 0.54205607 0.73239437 0.6713615  1.         0.97183099]
Accuracy: 0.7446717817561808
Recall:  0.7449

C:\Users\girid\anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [115]:
from sklearn.svm import SVC


In [116]:
def classify(data, label):
            
    
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
#     clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
    clf = SVC(kernel='linear', decision_function_shape='ovr')

    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]

    test_corpus = X_test
#     test_corpus = [" ".join(x) for x in test_corpus]

#     ax = axes.set_ylim([0,300])
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
#     # Instantiate the classification model and visualizer
#     visualizer = ClassPredictionError(clf, classes= ['human', 'ctrl', 'gpt', 'gpt2', 'grover', 'xlm', 'xlnet', 'pplm', 'fair'])

#     # Fit the training data to the visualizer
#     visualizer.fit(train_vector, y_train)

#     # Evaluate the model on the test data
#     visualizer.score(test_vector, y_test)

#     # Draw visualization
#     visualizer.show()
    
    l=['ctrl','fair' ,'gpt', 'gpt2', 'gpt3', 'grover', 'human', 'instructgpt', 'pplm', 'xlm', 'xlnet']
    matrix = confusion_matrix(y_test, pred, labels = l)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = l,
                                digits=4))
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

In [117]:
lr, v = classify(df['Generation'], df['label'])

              precision    recall  f1-score   support

        ctrl     1.0000    0.9953    0.9976       213
        fair     0.5425    0.6291    0.5826       213
         gpt     1.0000    1.0000    1.0000       213
        gpt2     0.5668    0.4953    0.5287       214
        gpt3     0.4873    0.6291    0.5492       213
      grover     0.6043    0.6636    0.6325       214
       human     0.7257    0.5935    0.6530       214
 instructgpt     0.6952    0.6854    0.6903       213
        pplm     0.9212    0.7136    0.8042       213
         xlm     0.9907    1.0000    0.9953       213
       xlnet     0.9953    0.9906    0.9929       213

    accuracy                         0.7630      2346
   macro avg     0.7754    0.7632    0.7660      2346
weighted avg     0.7752    0.7630    0.7658      2346

confusion matrix:  [0.99530516 0.62910798 1.         0.4953271  0.62910798 0.6635514
 0.59345794 0.68544601 0.71361502 1.         0.99061033]
Accuracy: 0.763000852514919
Recall:  0.763229

In [122]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\girid\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [127]:
from nltk import pos_tag, word_tokenize

X1 = []

for text in df['Generation']:
    tokens = word_tokenize(text)
    pos_tags = [[word,tag] for word, tag in pos_tag(tokens)]
    pos_features = {word:tag for [word,tag] in pos_tags}
    X1.append(pos_features)
#     y.append(label)

In [128]:
X[0]

{'JJ': 1,
 'NNP': 1,
 'VB': 1,
 'DT': 1,
 'NN': 1,
 'WDT': 1,
 'VBZ': 1,
 'TO': 1,
 'PRP': 1}

In [129]:
X1[0]

{'Local': 'JJ',
 'News': 'NNP',
 'get': 'VB',
 'the': 'DT',
 'news': 'NN',
 'that': 'WDT',
 "'s": 'VBZ',
 'local': 'JJ',
 'to': 'TO',
 'you': 'PRP'}

In [134]:
def clean_the_text(data):
    clean_data = []
    for i in list(data['Generation']):
        # Replace all the specified patterns with spaces
        text = re.sub('[^A-Za-z0-9]+|<unk>|\n| \ |<UNK>|News|<eos>|<eod>|<eop>', ' ', str(i))
        # Replace consecutive whitespaces with a single space
        text = re.sub('\s+', ' ', text)
        # Remove leading/trailing whitespaces
        text = text.strip()
        clean_data.append(text)
    
    # Create a new DataFrame with cleaned texts and original labels
    return clean_data

In [136]:
from nltk import word_tokenize

def get_pos(data):
    pos = []
    for i in data:
        pos.append( nltk.pos_tag(word_tokenize(i)))
    return pos
# clean_the_text

pos1 = get_pos(clean_the_text(df))


In [132]:
pos[0]

[('Local', 'JJ'),
 ('News', 'NNP'),
 ('get', 'VB'),
 ('the', 'DT'),
 ('news', 'NN'),
 ('that', 'WDT'),
 ("'s", 'VBZ'),
 ('local', 'JJ'),
 ('to', 'TO'),
 ('you', 'PRP')]

In [137]:
pos1[0]

[('Local', 'JJ'),
 ('get', 'VB'),
 ('the', 'DT'),
 ('news', 'NN'),
 ('that', 'IN'),
 ('s', 'VBZ'),
 ('local', 'JJ'),
 ('to', 'TO'),
 ('you', 'PRP')]

In [130]:
import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\girid\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


True

In [138]:
check_pos = []
for i in pos:
    words = nltk.FreqDist(tag for (word, tag) in i)
    check_pos.append(list(words.keys()))


In [139]:
check_pos[0]

['JJ', 'NNP', 'VB', 'DT', 'NN', 'WDT', 'VBZ', 'TO', 'PRP']

In [140]:
POS = [" ".join(x) for x in check_pos]


In [141]:
len(check_pos)

11726

In [142]:
pos_df = pd.DataFrame({'POS': POS, 'class': df['label']})

In [143]:
pos_df.head()

,POS,class
0,JJ NNP VB DT NN WDT VBZ TO PRP,human
1,": DT NN IN JJ NNS RB VBZ VBN CD . NNP , WRB VB...",instructgpt
2,"RB NNP WRB JJ NNS VBD PRP$ NN IN CC DT , MD VB...",pplm
3,"VBZ PRP , DT NN CC . WRB TO VB IN `` VBD JJ : ...",gpt
4,VBP RB VB DT NN ( PRP MD VBG PRP$ NNS ) : NNP ...,xlm


In [145]:
from sklearn.svm import SVC

def classify(data, label):
            
    
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
#     clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
    clf = SVC(kernel='linear', decision_function_shape='ovr')

    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]

    test_corpus = X_test
    
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    l=['ctrl','fair' ,'gpt', 'gpt2', 'gpt3', 'grover', 'human', 'instructgpt', 'pplm', 'xlm', 'xlnet']
    matrix = confusion_matrix(y_test, pred, labels = l)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = l,
                                digits=4))
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

lr, v = classify(pos_df['POS'], pos_df['class'])

              precision    recall  f1-score   support

        ctrl     0.4538    0.5540    0.4989       213
        fair     0.3699    0.3005    0.3316       213
         gpt     0.6770    0.7183    0.6970       213
        gpt2     0.4732    0.4953    0.4840       214
        gpt3     0.5263    0.3756    0.4384       213
      grover     0.2717    0.3505    0.3061       214
       human     0.1915    0.1262    0.1521       214
 instructgpt     0.4803    0.6291    0.5447       213
        pplm     0.3301    0.3192    0.3246       213
         xlm     0.7258    0.6338    0.6767       213
       xlnet     0.6099    0.6385    0.6239       213

    accuracy                         0.4672      2346
   macro avg     0.4645    0.4674    0.4616      2346
weighted avg     0.4643    0.4672    0.4615      2346

confusion matrix:  [0.55399061 0.30046948 0.71830986 0.4953271  0.37558685 0.35046729
 0.12616822 0.62910798 0.31924883 0.63380282 0.63849765]
Accuracy: 0.46717817561807334
Recall:  0.467

In [146]:
from sklearn.svm import SVC

def classify(data, label):
            
    
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)
#     clf = SVC(kernel='linear', decision_function_shape='ovr')

    v = TfidfVectorizer()
    
    train_corpus = X_train
#     train_corpus = [[word.lower() for word in text.split()] for text in data]

    test_corpus = X_test
    
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    l=['ctrl','fair' ,'gpt', 'gpt2', 'gpt3', 'grover', 'human', 'instructgpt', 'pplm', 'xlm', 'xlnet']
    matrix = confusion_matrix(y_test, pred, labels = l)
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, pred, labels = l,
                                digits=4))
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    print('F1:', F1)
    
    return clf, v

lr, v = classify(pos_df['POS'], pos_df['class'])

              precision    recall  f1-score   support

        ctrl     0.4570    0.5493    0.4989       213
        fair     0.2983    0.2535    0.2741       213
         gpt     0.6026    0.6620    0.6309       213
        gpt2     0.4395    0.4579    0.4485       214
        gpt3     0.4932    0.5117    0.5023       213
      grover     0.2959    0.2336    0.2611       214
       human     0.3964    0.3131    0.3499       214
 instructgpt     0.4894    0.5399    0.5134       213
        pplm     0.3486    0.3568    0.3527       213
         xlm     0.6385    0.6385    0.6385       213
       xlnet     0.5727    0.6103    0.5909       213

    accuracy                         0.4659      2346
   macro avg     0.4575    0.4661    0.4601      2346
weighted avg     0.4574    0.4659    0.4600      2346

confusion matrix:  [0.54929577 0.25352113 0.66197183 0.45794393 0.51173709 0.23364486
 0.31308411 0.5399061  0.35680751 0.63849765 0.61032864]
Accuracy: 0.4658994032395567
Recall:  0.4660